# Finetuning Base Model


In [1]:
%%bash
# install necessary libraries
pip install sentence_transformers torch tabulate langchain-openai evaluate scikit-learn transformers sentencepiece datasets accelerate nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 920.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

In [2]:
# import necessary libraries
import json
import pandas as pd
import os
import torch
from datasets import load_dataset,DatasetDict, Dataset
from transformers import pipeline,DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM, AutoTokenizer,TrainingArguments, Trainer

2024-08-16 09:26:16.594167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 09:26:16.594273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 09:26:16.758143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# set the gpu device as gpu
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
# define model checkpoint and load tokenizer and model
MODEL_CHECKPOINT = 'google-t5/t5-base'
# download data using https://drive.google.com/file/d/1tuiAVkfy_EVM0zavMcWPvqeuEGDdpbnG/view?usp=sharing
INPUT_DATA_PATH_2='/kaggle/input/preprocessedpubmed/preprocessed_pubmed_dataset.xls'


## Load Pubmed Dataset
Load data into a dataframe with article_id, article_text and article_text_raw as attributes

In [6]:
def load_data(path:str=INPUT_DATA_PATH_2)->pd.DataFrame:
    """
    Load Pubmed dataset to dataframe
    """
    pubmed_data={}
    article_id=""
    article_text=""
    print("############# Started Data Loading ##############")
    with open(path, 'r') as file:
        for line in file:
            article_id = json.loads(line)['article_id']
            article_text = " ".join(json.loads(line)['article_text'])
            abstract_text="".join(x.strip("<S> </S>") for x in json.loads(line)['abstract_text'])
            pubmed_data[article_id]=[article_text,abstract_text]
    df=pd.DataFrame.from_dict(pubmed_data,orient='index',columns=['article_text','abstract_text'])
    print("############# Finished Data Loading ##############")
    return df

## Preprocess Data to convert it from Dataframe to DatasetDict

In [11]:
import numpy as np
import torch
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer

class preprocess_data_pubmed:
    columns = ['input_ids', 'labels', 'attention_mask']
    TRAIN_RANGE = 15000
    VAL_RANGE = 5000
    TEST_RANGE = 5000
    MODEL_CHECKPOINT = 'google-t5/t5-base'  
    TOKENIZER = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

    def get_feature(self, batch):
        """
        Get encodings for all the input data
        """
        # Tokenize inputs
        encodings = self.TOKENIZER(batch['article_text'], text_target=batch['abstract_text'], max_length=512, truncation=True, padding='max_length')

        # Return encodings with input_ids, attention_mask, and labels
        return {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'labels': encodings['labels']
        }

    def map_data(self, pubmed_df):
        """
        Map the input data to be compatible with PyTorch format
        """
        data_dict = {
            "train": Dataset.from_dict({
                "article_text": pubmed_df['article_text'][:self.TRAIN_RANGE].tolist(),
                "abstract_text": pubmed_df['abstract_text'][:self.TRAIN_RANGE].tolist()
            }),
            "validation": Dataset.from_dict({
                "article_text": pubmed_df['article_text'][self.TRAIN_RANGE:self.TRAIN_RANGE + self.VAL_RANGE].tolist(),
                "abstract_text": pubmed_df['abstract_text'][self.TRAIN_RANGE:self.TRAIN_RANGE + self.VAL_RANGE].tolist()
            }),
            "test": Dataset.from_dict({
                "article_text": pubmed_df['article_text'][self.TRAIN_RANGE + self.VAL_RANGE:self.TRAIN_RANGE + self.VAL_RANGE + self.TEST_RANGE].tolist(),
                "abstract_text": pubmed_df['abstract_text'][self.TRAIN_RANGE + self.VAL_RANGE:self.TRAIN_RANGE + self.VAL_RANGE + self.TEST_RANGE].tolist()
            })
        }
        
        dataset_dict = DatasetDict(data_dict)
        pubmed_pt = dataset_dict.map(self.get_feature, batched=True)
        pubmed_pt.set_format(type='torch', columns=self.columns)
        
        return pubmed_pt


#used before-last part

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## Trainer Class

In [16]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class trainer_class:
    def __init__(self, pubmed_pt):
        """
        initialize arguments for training
        """
        self.MODEL_CHECKPOINT = 'google-t5/t5-base'
        self.MODEL = AutoModelForSeq2SeqLM.from_pretrained(self.MODEL_CHECKPOINT)
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_CHECKPOINT)
        self.DATA_COLLATOR = DataCollatorForSeq2Seq(self.TOKENIZER, model=self.MODEL)
        self.TRAINING_ARGS = TrainingArguments(
            output_dir=self.MODEL_CHECKPOINT,
            num_train_epochs=3,
            warmup_steps=500,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
#             gradient_accumulation_steps=16,
#             dataloader_num_workers=4,  # Increase the number of workers for data loading
#             report_to="none"  # Disable reporting to avoid additional overhead
        )
        self.PUBMED_PT = pubmed_pt

    def train_model(self):
        """
        train the model and save it locally
        """
        trainer = Trainer(
            model=self.MODEL,
            args=self.TRAINING_ARGS,
            tokenizer=self.TOKENIZER,
            data_collator=self.DATA_COLLATOR,
            train_dataset=self.PUBMED_PT['train'],
            eval_dataset=self.PUBMED_PT['validation']
        )
        trainer.train()
        trainer.save_model(self.MODEL_CHECKPOINT + '_pretrained_model')

## Start finetuning

In [20]:
%%time
pubmed_df=pd.read_csv(INPUT_DATA_PATH_2)

CPU times: user 19.5 s, sys: 2.64 s, total: 22.2 s
Wall time: 34.8 s


In [21]:
pubmed_df.isna().sum()

article     2574
abstract       0
dtype: int64

In [22]:
pubmed_df = pubmed_df.dropna(subset=['article'])

In [23]:
pubmed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107426 entries, 0 to 109999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   article   107426 non-null  object
 1   abstract  107426 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [24]:
print(pubmed_df.dtypes)

article     object
abstract    object
dtype: object


In [25]:
pubmed_df = pubmed_df.rename(columns={'article': 'article_text', 'abstract': 'abstract_text'})

In [26]:
pubmed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107426 entries, 0 to 109999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   article_text   107426 non-null  object
 1   abstract_text  107426 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [28]:
%%time
preprocess = preprocess_data_pubmed()
pubmed_pt = preprocess.map_data(pubmed_df)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

CPU times: user 5min 33s, sys: 3.91 s, total: 5min 37s
Wall time: 5min 37s


In [27]:
# %%time
# pubmed_pt = preprocess_data().map_data(pubmed_df)

In [29]:
# if torch.cuda.is_available():
#   generator = torch.Generator('cuda').manual_seed(seed)
# else:
#   generator = torch.Generator().manual_seed(seed)

In [30]:
%%time
trainer_class(pubmed_pt).train_model()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
500,1.405100,1.489818
1000,1.648100,1.429936
1500,1.346700,1.402751
2000,1.280500,1.389172
2500,1.596500,1.375329
3000,1.288800,1.366514
3500,1.350500,1.359404
4000,1.376700,1.352479
4500,1.354100,1.346703
5000,1.682400,1.342891


CPU times: user 6h 31min 11s, sys: 40min 1s, total: 7h 11min 13s
Wall time: 7h 19min 12s


9301a2974b346ad642f89a348c6dd249d5a1062d

In [31]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [32]:
download_file('/kaggle/working/wandb', 'wandb')

/kaggle/working/wandb.zip

In [35]:
# Specify the directory where the model is saved 
MODEL_CHECKPOINT = 'google-t5/t5-base'
saved_model_dir = MODEL_CHECKPOINT + '_pretrained_model'

# Load the model and tokenizer from the saved directory
model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)

# Example input text for summarization
input_text = "a recent systematic analysis showed that in 2011 314 296 331 million children younger than years were mildly moderately or severely stunted and 258 240 274 million were mildly moderately or severely underweight in the developing countries in iran study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% 8.6% and 1.5% for underweight overweight and obesity respectively the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight snack should have 300 400 kcal energy and could provide 10 of protein day nowadays school nutrition programs are running as the national programs world wide national school lunch program in the united states there are also some reports regarding school feeding programs in developing countries in vietnam school base program showed an improvement in nutrient intakes in iran national free food program nffp is implemented in elementary schools of deprived areas to cover all poor students however this program is not conducted in slums and poor areas of the big cities so many malnourished children with low socio economic situation are not covered by nffp although the rate of poverty in areas known as deprived is higher than other areas many students in deprived areas are not actually poor and can afford food hence nutritional value of the nffp is lower than the scientific recommended snacks for this age group furthermore lack of variety of food packages has decreased the tendency of children toward nffp on the other hand the most important one is ministry of education moe of iran which is responsible for selecting and providing the packages for targeted schools the ministry of health moh is supervising the health situation of students and their health needs welfare organizations along with charities have the indirect effect on nutritional status of students by financial support of their family provincial governors have also the role of coordinating and supervising all activities of these organizations parent teacher association is community based institution that participates in school policy such as nffp in addition to these organizations nutritional literacy of students their parents and teachers is very important issue which could affect nutritional status of school age children therefore the present study was conducted with the aim of improving the nffp so that by its resources all poor children will be covered even in big cities moreover all food packages were replaced by nutritious and diverse packages that were accessible for non poor children according to the aim of this study and multiple factors that could affect the problem public health advocacy has been chosen as the best strategy to deal with this issue therefore the present study determines the effects of nutrition intervention in an advocacy process model on the prevalence of underweight in school aged children in the poor area of shiraz iran this interventional study has been carried out between 2009 and 2010 in shiraz iran this survey was approved by the research committee of shiraz university of medical sciences in coordination with education organization of fars province two elementary schools and one middle school in the third region of the urban area of shiraz were selected randomly in those schools all students 2897 13 years old were screened based on their body mass index bmi by nutritionists according to convenience method all students divided to two groups based on their economic situation family revenue and head of household job and nutrition situation the first group were poor and malnourished students and the other group were well nourished or well off students for this report the children height and weight were entered into center for disease control and prevention cdc to calculate bmi and bmi for age scores based on cdc for diseases control and prevention and growth standards the significance of the difference between proportions was calculated using two tailed tests for independent proportions for implementing the interventions the advocacy process model weight was to the nearest 0.1 kg on balance scale model seca scale standing height was measured to the nearest 0.1 cm with wall mounted stadiometer advocacy group formation this step was started with stakeholder analysis and identifying the stakeholders the team was formed with representatives of all stakeholders include education organization welfare organization deputy for health of shiraz university food and cosmetic product supervisory office and several non governmental organizations and charities situation analysis this was carried out by use of existing data such as formal report of organizations literature review and focus group with experts the prevalence of malnutrition and its related factors among students was determined and weaknesses and strengths of the nffp were analyzed accordingly three sub groups were established research and evaluation education and justification and executive group designing the strategies three strategies were identified education and justification campaign nutritional intervention providing nutritious safe and diverse snacks and networking performing the interventions interventions that were implementing in selected schools were providing diverse and nutritious snack package along with nutrition education for both groups while the first group poor and malnourished students was utilized the package free of charge education and justification intervention regarding the literature review and expert opinion an educational group affiliated with the advocacy team has prepared educational booklets about nutritional information for each level degree accordingly education of these booklets has been integrated into regular education of students and they educated and justified for better nutrition life style it leads the educational group to hold several meeting with the student parents to justify them about the project and its benefit for their children after these meetings parental desire for participation in the project illustrated the effectiveness of the justification meeting with them for educate fifteen talk show programs in tv and radio 12 published papers in the local newspaper have implemented to mobilize the community and gain their support healthy diet the importance of breakfast and snack in adolescence wrong food habits among school age children role of the family to improve food habit of children were the main topics in which media campaign has focused on nutritional intervention the snack basket of the students was replaced with traditional nutritious and diverse foods in general the new snack package in average has provided 380 kcal energy 15 protein along with sufficient calcium and iron low economic and malnourished children were supported by executive group affiliated with advocacy team and the rest of them prepare their snack by themselves research and evaluation in this step the literacy and anthropometric indices bmi of students were assessed before and after the interventions the reference for anthropometric measures was the world health organization national center for health statistics who nchs standards and the cut offs were two standard deviations sd from the mean each student that was malnourished and poor has been taken into account for free food and nutritious snacks demographic information height weight and knowledge of the students were measured by use of validated and reliable cronbach alpha was 0.61 questionnaire this project is granted by shiraz university of medical sciences charities and welfare organization and education organization of fars province statistical analyses were performed using the statistical package for the social sciences spss software version 17.0 spss inc the results are expressed as mean sd and proportions as appropriated in order to determine the effective variables on the malnutrition status paired test was used to compare the end values with baseline ones in each group in this project the who score cut offs used were as follow using bmi for age scores overweight sd i.e score equivalent to bmi 25 kg obesity sd equivalent to bmi 30 kg thinness sd and severe thinness sd this interventional study has been carried out between 2009 and 2010 in shiraz iran this survey was approved by the research committee of shiraz university of medical sciences in coordination with education organization of fars province two elementary schools and one middle school in the third region of the urban area of shiraz were selected randomly in those schools all students 2897 13 years old were screened based on their body mass index bmi by nutritionists according to convenience method all students divided to two groups based on their economic situation family revenue and head of household job and nutrition situation the first group were poor and malnourished students and the other group were well nourished or well off students for this report the children height and weight were entered into center for disease control and prevention cdc to calculate bmi and bmi for age scores based on cdc for diseases control and prevention and growth standards the significance of the difference between proportions was calculated using two tailed tests for independent proportions for implementing the interventions weight was to the nearest 0.1 kg on balance scale model seca scale standing height was measured to the nearest 0.1 cm with wall mounted stadiometer advocacy group formation this step was started with stakeholder analysis and identifying the stakeholders the team was formed with representatives of all stakeholders include education organization welfare organization deputy for health of shiraz university food and cosmetic product supervisory office and several non governmental organizations and charities situation analysis this was carried out by use of existing data such as formal report of organizations literature review and focus group with experts the prevalence of malnutrition and its related factors among students was determined and weaknesses and strengths of the nffp were analyzed accordingly three sub groups were established research and evaluation education and justification and executive group designing the strategies three strategies were identified education and justification campaign nutritional intervention providing nutritious safe and diverse snacks and networking performing the interventions interventions that were implementing in selected schools were providing diverse and nutritious snack package along with nutrition education for both groups while the first group poor and malnourished students was utilized the package free of charge duration of intervention was months education and justification intervention regarding the literature review and expert opinion an educational group affiliated with the advocacy team has prepared educational booklets about nutritional information for each level degree accordingly education of these booklets has been integrated into regular education of students and they educated and justified for better nutrition life style obviously student families had remarkable effect on children food habit it leads the educational group to hold several meeting with the student parents to justify them about the project and its benefit for their children after these meetings parental desire for participation in the project illustrated the effectiveness of the justification meeting with them educate fifteen talk show programs in tv and radio 12 published papers in the local newspaper have implemented to mobilize the community and gain their support healthy diet the importance of breakfast and snack in adolescence wrong food habits among school age children role of the family to improve food habit of children were the main topics in which media campaign has focused on nutritional intervention the snack basket of the students was replaced with traditional nutritious and diverse foods in general the new snack package in average has provided 380 kcal energy 15 protein along with sufficient calcium and iron low economic and malnourished children were supported by executive group affiliated with advocacy team and the rest of them prepare their snack by themselves research and evaluation in this step the literacy and anthropometric indices bmi of students were assessed before and after the interventions the reference for anthropometric measures was the world health organization national center for health statistics who nchs standards and the cut offs were two standard deviations sd from the mean each student that was malnourished and poor has been taken into account for free food and nutritious snacks demographic information height weight and knowledge of the students were measured by use of validated and reliable cronbach alpha was 0.61 questionnaire this project is granted by shiraz university of medical sciences charities and welfare organization and education organization of fars province advocacy group formation this step was started with stakeholder analysis and identifying the stakeholders the team was formed with representatives of all stakeholders include education organization welfare organization deputy for health of shiraz university food and cosmetic product supervisory office and several non governmental organizations and charities situation analysis this was carried out by use of existing data such as formal report of organizations literature review and focus group with experts the prevalence of malnutrition and its related factors among students was determined and weaknesses and strengths of the nffp were analyzed accordingly three sub groups were established research and evaluation education and justification and executive group designing the strategies three strategies were identified education and justification campaign nutritional intervention providing nutritious safe and diverse snacks and networking performing the interventions interventions that were implementing in selected schools were providing diverse and nutritious snack package along with nutrition education for both groups while the first group poor and malnourished students was utilized the package free of charge education and justification intervention regarding the literature review and expert opinion an educational group affiliated with the advocacy team has prepared educational booklets about nutritional information for each level degree accordingly education of these booklets has been integrated into regular education of students and they educated and justified for better nutrition life style obviously student families had remarkable effect on children food habit it leads the educational group to hold several meeting with the student parents to justify them about the project and its benefit for their children after these meetings parental desire for participation in the project illustrated the effectiveness of the justification meeting with them educate fifteen talk show programs in tv and radio 12 published papers in the local newspaper have implemented to mobilize the community and gain their support healthy diet the importance of breakfast and snack in adolescence wrong food habits among school age children role of the family to improve food habit of children were the main topics in which media campaign has focused on nutritional intervention the snack basket of the students was replaced with traditional nutritious and diverse foods in general the new snack package in average has provided 380 kcal energy 15 protein along with sufficient calcium and iron low economic and malnourished children were supported by executive group affiliated with advocacy team and the rest of them prepare their snack by themselves research and evaluation in this step the literacy and anthropometric indices bmi of students were assessed before and after the interventions the reference for anthropometric measures was the world health organization national center for health statistics who nchs standards and the cut offs were two standard deviations sd from the mean each student that was malnourished and poor has been taken into account for free food and nutritious snacks demographic information height weight and knowledge of the students were measured by use of validated and reliable cronbach alpha was 0.61 questionnaire this project is granted by shiraz university of medical sciences charities and welfare organization and education organization of fars province statistical analyses were performed using the statistical package for the social sciences spss software version 17.0 spss inc chicago il usa the results are expressed as mean sd and proportions as appropriated in order to determine the effective variables on the malnutrition status paired test was used to compare the end values with baseline ones in each group two sided 0.05 was considered to be statistically significant in this project the who score cut offs used were as follow using bmi for age scores overweight sd i.e score equivalent to bmi 25 kg obesity sd equivalent to bmi 30 kg thinness sd and severe thinness sd study population contains 2897 children 70.8% were primary school students and 29.2% were secondary school students 2336 80.5% out of total students were well off and 561 children 19.5% were indigent 19.5% of subjects were in case group 561 and 80.5% were in the control group 2336 the mean of age in welfare group was 10.0 2.3 and 10.5 2.5 in non welfare group demographic characteristics of school aged children in shiraz iran table shows the frequency of subjects in different categories of bmi for age in non welfare and welfare groups of school aged children separately among boys and girls before and after nutrition intervention based on advocacy process model in shiraz iran the frequency of subjects with bmi lower than sd decreased significantly after intervention among non welfare girls 0.01 however there were no significant decreases in the frequency of subjects with bmi lower than sd boys when we assess the effect of intervention in total population without separating by sex groups we found no significant change in this population table bmi for age for iranian students aged 14 years based on gender according to who growth standards 2007 bmi for age for iranian students aged 14 years according to who growth standards 2007 in non welfare and welfare groups of total population table has shown the prevalence of normal bmi mild moderate and severe malnutrition in non welfare and welfare groups of school aged children separately among boys and girls before and after nutrition intervention based on advocacy process model according to this table there were no significant differences in the prevalence of mild moderate and severe malnutrition among girls and boys table also shows the mean of all anthropometric indices changed significantly after intervention both among girls and boys the pre and post test education assessment in both groups showed that the student average knowledge score has been significantly increased from 12.5 3.2 to 16.8 4.3 0.0001 bmi height and weight in non welfare and welfare groups of school aged children separately in males and females before and after nutrition intervention based on advocacy process model in shiraz iran according to study finding the odds ratio or of sever thinness and thinness in non welfare compared with welfare is 3.5 or 3.5 confidence interval ci 2.5 3.9 0.001 furthermore the finding showed or of overweight and obesity in welfare compared to non welfare is 19.3 or 19.3 ci 2.5 3.9 0.04 the result of this community intervention study revealed that nutrition intervention based on advocacy program had been successful to reduce the prevalence of underweight among poor girls this study shows determinant factor of nutritional status of school age children was their socio economic level according to our knowledge this is the first study which determines the effect of community intervention based on advocacy process on the malnutrition indices in big city shiraz in iran the other program in iran nffp is specified to deprived area and is not conducted in big cities allocating millions of dollars to nffp by government selecting the malnourished students through an active screening system at primary and middle schools paying attention of policy makers to student nutrition have provided the opportunity to combat the problem however negligence of under poverty line providing poor snacks in terms of nutritional value and lack of variety are the main defects of this program advocacy by definition is blending of science ethics and politics for comprehensive approaching health issues by using advocacy program in california among the high school students for improving their nutrition and physical activity angeles unified school district participants emphasized on nutrition classes for families as well as students in addition to other interventions in the present study another study revealed that evaluability assessment gave stakeholders the opportunity to reflect on the project and its implementation issues it seems that in iran free food program among the students not only is needed in deprived areas but also it should be performed in big cities such as shiraz at baseline no significant difference was founded among wealthy students between the pre and post nutritional status intervention in contrast the numbers of students who have malnutrition decreased from 44% to 39.4% which was identified as significant among impecunious girls students there was also significant increase in the proportion of children with bmi that was normal for age to sd most of the published community interventions showed better results among females compared with males this difference in the impact of nutritional interventions between male and female might be related to the different age of puberty in the female population compared to the male population in the age range of the present study female although there is no nffp in big cities of iran there are some programs for improving the nutritional status such as providing free milk in schools recent publication has shown that school feeding programs focus on milk supplementation had beneficial effects on the physical function and school performances specifically among girls in iran the results of the mentioned study showed an improvement in the weight of children psychological test scores and the grade point average following this school feeding program the intervention in the present study had focused on the snack intake in the school time there are some reports regarding the nutrition transition in iran which shows the importance of nutrition intervention to provide more healthy eating dietary habits among welfare groups of adolescents hence nutrition intervention especially in the form of nutrition education is needed in big cities and among welfare children and adolescents although study among iranian adolescents showed that dietary behavior of adolescents does not accord to their knowledge which emphasize on the necessity of community intervention programs recent study regarding the major dietary pattern among iranian children showed the presence of four major dietary patterns in which fast food pattern and sweet pattern as two major dietary patterns can be mentioned among iranian children in advocacy program audience analysis accordingly one of the prominent strategies in this study was working with media and was meeting with parent teacher association that both of them were secondary target audiences we also took into account policy makers in different levels from national to local as primary audiences advocacy team had several meetings with management and planning organization at national level and education organization of the fars province as well as principal of the targeted schools providing nutritious snacks need contribution of private sector such as food industries or factories but their benefits should be warranted another choice was community involvement which can be achieved by female health volunteers who are working with the health system advocacy team by using the support of charities and female health volunteers could establish local factory that produced student snacks based on the new definition however there are some challenges on the way of expanding this program mass production of the proposed snacks according to different desires and cultures and getting involvement of food industries with respect to marketing issues is one of those challenges moreover providing supportive environment in order to change the food habits of the students and their parents among the wide range of the population require sustainable and continuous inter sector collaboration although in limited number of schools in our study interventions and advocacy program was successful expanding this model to another areas around the country depends on convincing the policy makers at national level in this regard advocacy team should prepare evidenced based profile and transitional planning to convince the policy makers for improving the rule and regulation of nffp the same as this study in other studies have also emphasized that there must be efforts to strengthen the capacity within the schools to deal with the nutritional problems either overweight obesity or malnutrition by using of educational and nutritional intervention assessing the dietary adherence is very important in nutrition intervention among population as this population was children and adolescents we had limitation in the blood sample collection to assess the subject dietary adherence furthermore this intervention was only focused on the intake of snack in school time and we did not have comprehensive information on the dietary intake of children and adolescents after school all over the day the investigators propose further investigation in different areas of the country based on socio cultural differences in order to make necessary modification and adapt this model to other areas regarding the nutritional needs of the school age children provision of good platform for implementing and expanding this efficient model to the whole country based upon the socio economic situation of each region is advisable to the moh and the moe community nutrition intervention based on the advocacy process model is effective on reducing the prevalence of underweight specifically among female school aged children."

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=200, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:", summary)


Generated Summary: background national free food program nffp is implemented in elementary schools of deprived areas to cover all poor children however this program is not conducted in slums and poor areas of big cities so many malnourished children with low socio economic situation are not covered by nffp although the rate of poverty in areas known as deprived is higher than other areas many students in deprived areas are not actually poor and can afford food hence lack of variety of food packages has decreased the tendency of children toward nffp furthermore lack of variety of food packages has decreased the tendency of children toward nffp therefore the present study was conducted with the aim of improving the nffp so that by its resources all poor children will be covered even in big cities moreover all food packages were replaced with nffp so that by its resources all poor children will be covered even in big cities moreover all food packages were replaced with 
